<a href="https://colab.research.google.com/github/tmyok/Colaboratory/blob/master/VSCode_SSH_to_GoogleColab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google Colab をSSH と VS Code で使う手順のメモ

## 参考リンク
Google Colab をSSH と VS Code で使う  
https://qiita.com/hazigin/items/c291adf5dc9ccc13d11f  
How can I ssh to google colaboratory VM?  
https://stackoverflow.com/questions/48459804/how-can-i-ssh-to-google-colaboratory-vm/63186681#63186681  
NGROK  
https://ngrok.com  

## 下準備
Colab のランタイムをGPUに変更しておく

## Colab側の設定
下記を実行して、NRGOKからトークンを取得する

In [ ]:
import random, string, urllib.request, json, getpass

#Generate root password
password = ''.join(random.choice(string.ascii_letters + string.digits) for i in range(20))

#Download ngrok
! wget -q -c -nc https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
! unzip -qq -n ngrok-stable-linux-amd64.zip

#Setup sshd
! apt-get install -qq -o=Dpkg::Use-Pty=0 openssh-server pwgen > /dev/null

#Set root password
! echo root:$password | chpasswd
! mkdir -p /var/run/sshd
! echo "PermitRootLogin yes" >> /etc/ssh/sshd_config
! echo "PasswordAuthentication yes" >> /etc/ssh/sshd_config
! echo "LD_LIBRARY_PATH=/usr/lib64-nvidia" >> /root/.bashrc
! echo "export LD_LIBRARY_PATH" >> /root/.bashrc

#Run sshd
get_ipython().system_raw('/usr/sbin/sshd -D &')

#Ask token
print("Copy authtoken from https://dashboard.ngrok.com/auth/your-authtoken")

## 取得したトークンの入力
下記、"Yout Authtoken"の欄に、取得したトークンをペーストする

In [ ]:
authtoken = "Your Authtoken"

#Create tunnel
get_ipython().system_raw('./ngrok authtoken $authtoken && ./ngrok tcp 22 &')

#Get public address and print connect command
with urllib.request.urlopen('http://localhost:4040/api/tunnels') as response:
  data = json.loads(response.read().decode())
  (host, port) = data['tunnels'][0]['public_url'][6:].split(':')
  print(f'SSH command: ssh -p{port} root@{host}')

#Print root password
print(f'Root password: {password}')

## SSH接続
うまくいっていればSSH commandとRoot passwordが表示されるので、
~/.ssh/configを編集してVSCodeを設定する。